# Slackbot Example

SlackBot keeps you in the loop without disturbing your focus. Its personalized, intelligent AI continuously monitors your Slack workspace, alerting you to important conversations and freeing you to concentrate on what’s most important.

SlackBot reads the full history of your (public) Slack workspace and trains a Generative AI model to predict when you need to engage with a conversation. This training process gives the AI a deep understanding of your interests, expertise, and relationships. Using this understanding, SlackBot watches conversations in real-time and notifies you when an important conversation is happening without you. With SlackBot you can focus on getting things done without worrying about missing out.

In this notebook, you’ll see you how to build and deploy SlackBot in 15 minutes using OpenAI’s API’s and open-source Python libraries - Data Science PhD not required.



In [ ]:
%pip install openai kaskada

In [ ]:
from datetime import datetime, timedelta
from slack_sdk.socket_mode import SocketModeClient, SocketModeResponse
import sparrow_pi as kt
import openai
import getpass
import pyarrow

# Initialize Kaskada with a local execution context.
kt.init_session()

# Initialize OpenAI
openai.api_key = getpass.getpass('OpenAI: API Key')

# Initialize Slack
slack = SocketModeClient(
    app_token=getpass.getpass('Slack: App Token'),
    web_client=getpass.getpass('Slack: Bot Token'),
)

## Prompt Engineering

In [ ]:
def build_conversation(messages):
    message_time = messages.col("ts")
    last_message_time = message_time.lag(1) # !!!
    is_new_conversation = message_time.seconds_since(last_message_time) > 10 * 60

    return messages \
        .select("user", "ts", "text", "reactions") \
        .collect(window=kt.windows.Since(is_new_conversation), max=100)

In [ ]:
def build_examples(messages):
    duration = datetime.timedelta(minutes=5)

    coverstation = build_conversation(messages)
    shifted_coversation = coverstation.shift_by(duration)  # !!!

    reaction_users = coverstation.col("reactions").col("name").collect(kt.windows.Trailing(duration)).flatten()  # !!!
    participating_users = coverstation.col("user").collect(kt.windows.Trailing(duration))  # !!!
    engaged_users = kt.union(reaction_users, participating_users)  # !!!

    return kt.record({ "prompt": shifted_coversation, "completion": engaged_users}) \
        .filter(shifted_coversation.is_not_null())

## Fine-tune the model

In [ ]:
import pandas
import sparrow_pi.sources as sources

messages = kt.sources.Parquet("./messages.parquet", time = "ts", entity = "channel")
messages = messages.with_key(kt.record({  # !!!
        "channel": messages.col("channel"),
        "thread": messages.col("thread_ts"),
    }))
examples = build_examples(messages)

examples_df = examples.run().to_pandas()

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(examples_df.completion.explode())

# Format for the OpenAI API
def format_prompt(prompt):
    return "start -> " + "\n\n".join([f' {msg["user"]} --> {msg["text"]} ' for msg in prompt]) + "\n\n###\n\n"
examples_df.prompt = examples_df.prompt.apply(format_prompt)

def format_completion(completion):
    return " " + (" ".join([le.transform(u) for u in completion]) if len(completion) > 0 else "nil") + " end"
examples_df.completion = examples_df.completion.apply(format_completion)

# Write examples to file
examples_df.to_json("examples.jsonl", orient='records', lines=True)

In [ ]:
from types import SimpleNamespace
from openai import cli

# verifiy data format, split for training & validation
args = SimpleNamespace(file='./examples.jsonl', quiet=True)
cli.FineTune.prepare_data(args)
training_id = cli.FineTune._get_or_upload('./examples_prepared_train.jsonl', True)

In [ ]:
import openai

resp = openai.FineTune.create(
    training_file = training_id,
    model = "davinci",
    n_epochs = 2,
    learning_rate_multiplier = 0.02,
    suffix = "coversation_users"
)
print(f'Fine-tuning model with job ID: "{resp["id"]}"')

## Notify users of conversations they need to know about

In [ ]:
import json, math

min_prob_for_response = 0.50

# Receive Slack messages in real-time
live_messages = kt.sources.ArrowSource(entity_column="channel", time_column="ts")

# Receive messages from Slack
def handle_message(client, req):
    # Acknowledge the message back to Slack
    client.send_socket_mode_response(SocketModeResponse(envelope_id=req.envelope_id))
    
    if req.type == "events_api" and "event" in req.payload:
        e = req.payload["event"]
        if "previous_message" in e or  e["type"] == "reaction_added":
            return
        # send message events to Kaskada
        live_messages.add_event(pyarrow.json.read_json(e))

slack.socket_mode_request_listeners.append(handle_message)
slack.connect()

# Handle messages in realtime
# A "conversation" is a list of messages
for conversation in build_conversation(live_messages).start().to_generator():
    if len(conversation) == 0:
        continue
    
    # Ask the model who should be notified
    res = openai.Completion.create(
        model="davinci:ft-personal:coversation-users-full-kaskada-2023-08-05-14-25-30", 
        prompt=format_prompt(conversation),
        logprobs=5,
        max_tokens=1,
        stop=" end",
        temperature=0,
    )

    users = []
    logprobs = res["choices"][0]["logprobs"]["top_logprobs"][0]
    for user in logprobs:
        if math.exp(logprobs[user]) > min_prob_for_response:
            user = users.strip()
            # if users include `nil`, stop processing
            if user == "nil":
                users = []
                break
            users.append(user)

    # alert on most recent message in conversation
    msg = conversation.pop()
    
    # Send notification to users
    for user in users:
        user_id = le.inverse_transform(user)

        # get user channel for slackbot
        app = slack.web_client.users_conversations(
            types="im",
            user=user_id,
        )
        
        # confirm user has slackbot installed
        if len(app["channels"]) == 0:
            continue

        link = slack.web_client.chat_getPermalink(
            channel=msg["channel"],
            message_ts=msg["ts"],
        )["permalink"]
        
        slack.web_client.chat_postMessage(
            channel=app["channels"][0]["id"],
            text=f'You may be interested in this converstation: <{link}|{msg["text"]}>'
        )